# Construction du modèle de reconnaissance de caractère basé sur le dataset MIST

Dans ce notebook sera présenté la création du modèle qui permettra de **classifier** les lettres de l'alphabet de a-z et A-Z.

Pour ce faire nous allons utiliser le dataset d'images prétraités créés par le notebook [NIST-preprocessing](/notebooks/notebooks/character_recognition/NIST-preprocessing.ipynb)

Pour exécuter ce notebook, veillez à ce que le jeu de données soit bien sous la forme suivante.
- data
    - processed
         - NIST-dataset
            - train
                - a000001.png
                - a000002.png
                ...
                ...
                - a00000n.png
            - test_set
                - a000001.png
                - a000002.png
                ...
                ...
                - a00000n.png


## Importation des dépendances

In [1]:
import os
import datetime
import multiprocessing as mp
from string import ascii_lowercase, ascii_uppercase

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2023-04-21 02:04:37.776240: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 02:04:38.120861: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/alexandre/Programmes/Python-3.6.4/
2023-04-21 02:04:38.120896: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-21 02:04:39.525329: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: can

## Sélection des devices

In [2]:
from tensorflow.python.client import device_lib

print("Num of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_lib.list_local_devices()

Num of GPUs Available:  0


2023-04-21 02:04:40.721471: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/alexandre/Programmes/Python-3.6.4/
2023-04-21 02:04:40.721722: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-21 02:04:40.721741: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (alexandre-HP-Pavilion-Laptop-14-ec0xxx): /proc/driver/nvidia/version does not exist
2023-04-21 02:04:40.723083: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate c

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17300069916710730510
 xla_global_id: -1]

In [3]:
tf.test.is_built_with_cuda()
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

## Initialisation de tensorboard

In [4]:
%load_ext tensorboard

## Définition des paramètres du notebook

In [5]:
NIST_PROCESSED_PATH = "../../data/processed/NIST-dataset"
TRAIN_SET_PATH = os.path.join(NIST_PROCESSED_PATH, "train", "nist_processed_train.csv")
TEST_SET_PATH = os.path.join(NIST_PROCESSED_PATH, "test", "nist_processed_test.csv")
IMG_SIZE = 41
CHUNK_SIZE = 25000
PANDA_CSV_ENGINE = "c"

## Importation du dataset

Dans cette section, nous allons charger le dataset afin d'obtenir un train_data_set, train_label_set, ainsi qu'un test_data_set et un test_label_set

### Chargement du dataset depuis le disque

In [6]:
def get_y_original_shape(y_raw_set):
    x  = y_raw_set.shape
    return y_raw_set.reshape(x[0], 1)

def get_x_original_shape(x_raw_set):
    x, y = x_raw_set.shape
    return x_raw_set.reshape(x, IMG_SIZE, IMG_SIZE)

In [7]:
def get_y_and_x_set_from_dataframe(dataframe):
    y_raw_set = dataframe["label"].to_numpy(dtype=str)
    x_raw_set = dataframe.iloc[:, 2:].to_numpy(dtype=np.uint8)

    y_set = get_y_original_shape(y_raw_set)
    x_set = get_x_original_shape(x_raw_set)

    return y_set, x_set

In [8]:
def get_x_y_from_csv_multi_process(path, engine, chunk_size):
    chunks = pd.read_csv(path, chunksize=chunk_size, engine=engine)
    cpu_count = mp.cpu_count()

    with mp.Pool(cpu_count) as pool:
        data_list = [data for data in pool.map(get_y_and_x_set_from_dataframe, chunks)]

    data = list(zip(*data_list))
    y_set = np.concatenate(data[0])
    x_set = np.concatenate(data[1])

    return y_set, x_set


In [9]:
y_train, x_train = get_x_y_from_csv_multi_process(TRAIN_SET_PATH, PANDA_CSV_ENGINE, CHUNK_SIZE)
y_test, x_test = get_x_y_from_csv_multi_process(TEST_SET_PATH, PANDA_CSV_ENGINE, CHUNK_SIZE)

print(y_train.shape)
print(x_train.shape)
print(y_test.shape)
print(x_test.shape)

(287911, 1)
(287911, 41, 41)
(123391, 1)
(123391, 41, 41)


## Encodage des labels

On récupère tous les labels possibles

In [10]:
label_list = [char for char in ascii_lowercase + ascii_uppercase]
print(label_list[0:5])

['a', 'b', 'c', 'd', 'e']


On définit l'encoder à utiliser

In [11]:
encoder = LabelBinarizer()
encoder.fit(label_list)

LabelBinarizer()

On encode nos données au format OneShot

In [12]:
y_encoded_train = encoder.transform(y_train)
y_encoded_test = encoder.transform(y_test)
y_encoded_train.shape

(287911, 52)

## Création du modèle de classification

In [13]:
model = keras.Sequential([
    keras.Input(shape=(41,41)),
    layers.Flatten(input_shape=(41, 41)),
    layers.Dense(200, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(200, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(200, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(200, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(52, activation="softmax"),
])

model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["acc", "categorical_crossentropy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 1681)              0         
                                                                 
 dense (Dense)               (None, 100)               168200    
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 100)               10100     
                                                                 
 dense_4 (Dense)             (None, 52)                5252      
                                                                 
Total params: 203,752
Trainable params: 203,752
Non-trai

## Configuration de tensorboard si installé

In [14]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

## Entrainement du modèle

In [15]:
model.fit(x=x_train, y=y_encoded_train, epochs=20, callbacks=[tensorboard_callback])

2023-04-21 02:05:29.124403: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 483978391 exceeds 10% of free system memory.


Epoch 1/20
8998/8998 [==============================] - 23s 2ms/step - loss: 1.7573 - acc: 0.5228 - categorical_crossentropy: 1.7573
Epoch 2/20
8998/8998 [==============================] - 22s 2ms/step - loss: 0.9881 - acc: 0.7054 - categorical_crossentropy: 0.9881
Epoch 3/20
8998/8998 [==============================] - 21s 2ms/step - loss: 0.8228 - acc: 0.7468 - categorical_crossentropy: 0.8228
Epoch 4/20
8998/8998 [==============================] - 22s 2ms/step - loss: 0.7400 - acc: 0.7687 - categorical_crossentropy: 0.7400
Epoch 5/20
8998/8998 [==============================] - 23s 3ms/step - loss: 0.6883 - acc: 0.7816 - categorical_crossentropy: 0.6883
Epoch 6/20
8998/8998 [==============================] - 23s 3ms/step - loss: 0.6520 - acc: 0.7907 - categorical_crossentropy: 0.6520
Epoch 7/20
8998/8998 [==============================] - 24s 3ms/step - loss: 0.6239 - acc: 0.7979 - categorical_crossentropy: 0.6239
Epoch 8/20
8998/8998 [==============================] - 23s 3ms/step 

## Test du modèle sur le jeu de test

In [16]:
model.evaluate(x_test, y_encoded_test, verbose=2)

3856/3856 - 8s - loss: 0.7403 - acc: 0.7694 - categorical_crossentropy: 0.7403 - 8s/epoch - 2ms/step


[0.7402697801589966, 0.7694321274757385, 0.7402697801589966]

In [17]:
%tensorboard --logdir {log_dir}

Launching TensorBoard...

## Sauvegarde du modèle

In [18]:
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets
